Оригинальный формат

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* <s>dob_years</s> age_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* <s>total_income</s> monthly_income — ежемесячный доход
* purpose — цель получения кредита

Созданные мной столбцы

* purpose_object_id - id цели кредита (существительное)
* purpose_action_id - id цели кредита (глагол)

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [20]:
import pandas as pd
raw_data = pd.read_csv('/datasets/data.csv')

from pymystem3 import Mystem # здесь же сразу все необходимые импорты
m = Mystem()

#from collections import Counter

print (raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

>В "days_employed" и "total_income" данных меньше чем в остальных столбцах, но одинаковое количество. Что логично: нет работы - нет дохода. Возможно человек не смог предоставить справку 

>Не понятно почему тип данных в days_employed float. По идее должен быть int. Не может быть информации что кто-то работает 100 дней и 7 часов, поэтому количество дней должно быть целым. И уж точно не отрицательным! <b>ПОСМОТРЕТЬ ЧТО ЭТО. ВОЗМОЖНО ПОПРАВИТЬ</b>

>Название столбца dob_years не очень понятно. dob это все таки дата рождения а не возраст, а данные в столбце именно возраст <b>ПЕРЕИМЕНОВАТЬ В age_years</b>

>Явная ошибка с регистром в столбце education - среднее & Среднее <b>ПРОВЕРИТЬ ВСЕ ЗАПИСИ И ПОПРАВИТЬ</b>

>Не очень понятно зачем education если есть education_id (тоже самое относится к family_status vs family_status_id) В рамках того, что мы изучани, эти данные нужно <b>выносить в словари</b> хотя по хорошему список фиксированных значений можно просто включить в описание данных оставив в данных только id и на этом успокоиться. 

>В  total_income тоже что-то странное. Во-первых это не total_income а monthly_income Даже представить не могу откуа бы взялись десятитысячные доли копеек. Требуется уточнение у поставщика данных и у потребителя анализа насколько нужна эта информация с ТАКОЙ точностью, но пока предлагаю <b>округлить до рублей</b> и, соответственно, преобразовать в int

### Шаг 2. Предобработка данных

>Сначала переименуем столбец dob_years -> age_years и total_income в monthly_income

In [21]:
raw_data.set_axis(['children', 'days_employed', 'age_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'monthly_income', 'purpose'], axis = 'columns', inplace = True)


print (raw_data.columns) #проверяем что получилось

Index(['children', 'days_employed', 'age_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'monthly_income', 'purpose'],
      dtype='object')


### Обработка пропусков

In [22]:
print (raw_data.isnull().sum()) # смотрим что там у нас с пропусками

children               0
days_employed       2174
age_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
monthly_income      2174
purpose                0
dtype: int64


### Вывод

>На первый взгляд все в порядке - пропуски там где нужно, по крайней мере ожидаемые. Проверю совпадают ли записи от отсутствии опыта работы и дохода на 100%. Удовлетворяющих обоим критериям записей должно быть 2174

In [23]:
print (raw_data[(raw_data['days_employed'].isna()) & raw_data['monthly_income'].isna()].shape[0])

2174


>Проверка прошла, заполняем все NaNы в days_employed и monthly_income на 0. Это никак не повлияет на анализ, но после этого мы сможем преобразовать оба столбца в int. Позже рассмотрю возможность заполнить пропуски чем то более осмысленным

>Перед заполнение пропусков нулями нужно проверить есть ли 0 уже. Не могу точно сказать зачем это, но как то мне кажется 
правильным отличать нули которые были изначально от нулей заполнителей.

In [24]:
print ('Записи с нулевым доходом', raw_data[(raw_data['monthly_income']==0)]['monthly_income'].count()) 
print ('Записи с нулевым стажем', raw_data[(raw_data['days_employed']==0)]['days_employed'].count())

raw_data['days_employed'] = raw_data['days_employed'].fillna(0) 
raw_data['monthly_income'] = raw_data['monthly_income'].fillna(0)

Записи с нулевым доходом 0
Записи с нулевым стажем 0


#### Анализ children

>Здесь нужно чтобы не было значенй меньше 0 и слишком большого их количества. То что нет дробных детей мы уже поняли - children non-null int64. Ну и чтобы не было пропусков

In [25]:
print ('Детей меньше нуля' , raw_data[raw_data['children'] < 0]['children'].count())
print ('Детей максимум' , raw_data[raw_data['children'] >0]['children'].max())
print ('Пропуски' , raw_data[(raw_data['children'].isna())].shape[0])


Детей меньше нуля 47
Детей максимум 20
Пропуски 0


>20? Хм... Такое конечно возможно, но на всякий случай сделаем дополнительную проверку

In [26]:
print (raw_data['children'].sort_values(ascending=False)) # Смотрим самые многотелные семьи

15812    20
16795    20
15313    20
5020     20
3302     20
         ..
11617    -1
20393    -1
6385     -1
9574     -1
15435    -1
Name: children, Length: 21525, dtype: int64


>Ого! 20 детей оказывается не 1-2 случая. Их много!

In [27]:
print(raw_data[raw_data['children'] == 20]['children'].count()) # Смотрим сколько семей есть с 20 детьми

76


>76 20-ти детных семей.... А что за ними?

In [28]:
print (raw_data['children'].sort_values(ascending=False)[75:84]) # Смотрим сколько детей после рекордсменов

19271    20
16211     5
15822     5
20452     5
21156     5
20837     5
15916     5
3979      5
4397      5
Name: children, dtype: int64


>76 записей с 20 детьми это очень сомнительно. Тем более что дальше сразу идут 5 детей. Плюс 47 записей с отрицательным числом. Итого 123 записи. Меньше 1 процента явного брака. И вот тпереь есть варианты:

* Обнуляем эти записи и пишем в рекомендации - нам не известно есть ли дети у этих людей или нет. Если мы их обнулим, то переведем в категорию "бездетных" и это может попортить нам анализ
* Заполняем какими то средними значениями - это совсем не подойдет. Опять же искажение статистики
* Удаляем записи - как не печально, но видимо это единственный вариант в данном случае. Данные критичные для задачи, и лучше вообще никаких чем искаженные. Тем более что достоверно восстановить их не получится

>Хотя... наверное стоит взглянуть на эти записи, а потом удалить ;) 

In [29]:
#print ('Детей меньше нуля' , raw_data[raw_data['children'] < 0].head(20))
#print ('Детей 20' , raw_data[raw_data['children'] ==20].head(20))


>Ну да, никакой четкой зависимости, по которой можно было бы определить точные данные. Так что удаляем!

In [30]:
raw_data = raw_data.loc[raw_data['children'] >= 0]
raw_data = raw_data.loc[raw_data['children'] != 20]


print ('Детей меньше нуля' , raw_data[raw_data['children'] < 0]['children'].count()) #проверка
print ('Детей максимум' , raw_data[raw_data['children'] >0]['children'].max())

Детей меньше нуля 0
Детей максимум 5


#### Анализ days_employed

In [31]:
print (raw_data.loc[:,'children':'age_years'].head(5)) # общий взгляд

   children  days_employed  age_years
0         1   -8437.673028         42
1         1   -4024.803754         36
2         0   -5623.422610         33
3         3   -4124.747207         32
4         0  340266.072047         53


>340266 дней это очень много, даже для 53 летнего человека! А ну как это часы? 340 266/8 = 42 533 все равно многовато. Очевидно 
это не единичная ошибка, т.к. дальше там еще бОльшие значения идут и не в единичном варианте. А если посмотреть на возраста тех, кто в положительной и отрицательной зоне?   

In [32]:
print (raw_data[raw_data['days_employed']<0]['age_years'].mean())
print (raw_data[raw_data['days_employed']>0]['age_years'].mean())

39.81959643241192
59.13523754007578


>Хм... попробуем так...

In [33]:
print (raw_data[raw_data['days_employed']<0]['age_years'].max())
print (raw_data[raw_data['days_employed']>0]['age_years'].min())

75
0


>Ну хоть какая то польза - нашелся новорожденный у которого стаж больше 0... А если так:

In [34]:
print (raw_data[raw_data['days_employed'] > 0]['income_type'].value_counts())
print (raw_data[raw_data['days_employed'] < 0]['income_type'].value_counts())
print (raw_data[raw_data['days_employed'] == 0]['income_type'].value_counts())

пенсионер      3429
безработный       2
Name: income_type, dtype: int64
сотрудник          9949
компаньон          4550
госслужащий        1307
студент               1
предприниматель       1
в декрете             1
Name: income_type, dtype: int64
сотрудник          1101
компаньон           504
пенсионер           410
госслужащий         146
предприниматель       1
Name: income_type, dtype: int64


>Вот. Что-то! Итак, у тех у кого стаж больше нуля есть дата окончания стажа. У все остальных "по сегодняшний день" Посмотрим
внимательно на тех у кого стаж < 0: 

In [35]:
print (raw_data[raw_data['days_employed']<0]['days_employed'].max())
print (raw_data[raw_data['days_employed']<0]['days_employed'].min())

-24.14163324048118
-18388.949900568383


>От 1 месяца до 50 лет. Реальные цифры. На всякий случай...

In [75]:
print (raw_data[raw_data['days_employed']==-18388.949900568383]['age_years']) # возраст того у кого стаж 50 лет

16202    61
Name: age_years, dtype: int64


>Т.е. стаж начал начисляться с 11 лет. Вроде невозможно. Но! Есть случаи, когда стаж начисляется в 1,5, двойном и даже тройном размере. Так что ничего невозможного (Чернобыль, лепризорий, крайний север и т.д.) Для более точной проверки сделаем отдельную проверку по соотношению стаж/(возраст - 16 лет) предполагая что стаж начисляется не раньше 16

In [77]:
check_employ_years= raw_data['days_employed'] * (-1)/365.25 #считаем года
employ_age = raw_data['age_years'] - 16 # считаем максимально возможный срок начисления стажа
check_employ_ratio = check_employ_years / employ_age 

check_employ_data = pd.DataFrame()

check_employ_data['years_employed'] = check_employ_years
check_employ_data['employ_age'] = employ_age
check_employ_data['check_employ_ratio'] = check_employ_ratio

print (check_employ_data.sort_values(by='check_employ_ratio', ascending = False).head(20))

       years_employed  employ_age  check_employ_ratio
14400    -1097.857291         -16           68.616081
577      -1089.271910         -16           68.079494
16725    -1083.898189         -16           67.743637
14545    -1081.312375         -16           67.582023
10120    -1017.564076         -16           63.597755
1167     -1015.412753         -16           63.463297
1889     -1013.400512         -16           63.337532
8010     -1003.306975         -16           62.706686
6994     -1002.238962         -16           62.639935
15759     -974.245817         -16           60.890364
12634     -972.237442         -16           60.764840
99        -948.779244         -16           59.298703
20278     -927.405526         -16           57.962845
4894      -921.330612         -16           57.583163
11975     -909.473934         -16           56.842121
20393     -908.258101         -16           56.766131
18953     -907.757840         -16           56.734865
3399        15.534202       

>Запомним что есть 17 младенцев со стажем около 1000 лет. Посмотрим позже, а пока посмотрим тех кто постарше

In [38]:
print (check_employ_data[check_employ_data['employ_age']>0].sort_values(by='check_employ_ratio', ascending = False).head(90))


del check_employ_data # подчищаем не нужный уже ДатаФрейм

       years_employed  employ_age  check_employ_ratio
3417        15.544842          13            1.195757
11066       10.675904           9            1.186212
5708        36.193110          31            1.167520
18789       11.648501          10            1.164850
13531       26.747507          23            1.162935
...               ...         ...                 ...
1539        43.248435          43            1.005778
4300        19.080962          19            1.004261
5995        16.059309          16            1.003707
13883       25.089888          25            1.003596
8           18.985932          19            0.999260

[90 rows x 3 columns]


>По идее все, что больше 1 уже подозрительно. Но, уже 90 запись опускается ниже 1 (т.к. данные о возрасте поставляются в годах, а о стаже в днях то расходнение может быть больше) Плюс в копилку теории что отрицательный стаж это положительный но посчитанный в другую сторону - максимальное значение ratio ~1.2 что не так много учитывая возможности двойных и тройных стажей. Т.е. вполне возможно что это реальные данные 

>Попробую разобраться с положительными значениями. Итак, я знаю что в положительных у меня:

>>пенсионер      3443
>>безработный       2

>Есть еще люди с нулевым стажем:

>>сотрудник          1105
>>компаньон           508
>>пенсионер           413
>>госслужащий         147
>>предприниматель       1

>Посмотрим сначала на положительные

In [78]:
print (raw_data[raw_data['days_employed']>0]['days_employed'].min())
print (raw_data[raw_data['days_employed']>0]['days_employed'].max())

328728.72060451825
401755.40047533


>Если это правда дни, то прикинем в годах: от 900 до 1100 (делил на 365,25) 200 лет слишком большая разница чтобы быть правдой. А если поделить еще на 30? Ну скажем года (из стравки пенсионного фонда, там стаж в годах - 22,97 у меня например) планировали умножать на 12 а потом на 30, но потом решили умножать на 365 (или 365,25) а про 30 забыли...

>Получается от 30 до 36,7.... "Средний трудовой стаж в РФ при выходе на пенсию составляет 34,5 года" - это данные за 2013 год. Уже похоже на правду!

>Кстати, средняя пенсия за январь 2018 года (Росстат) - 13323. Запомним на будущее!

>Посмотрим что там с двумя безработными у которых тоже очень большой стаж

In [79]:
print (raw_data[raw_data['income_type']=='безработный'])

Empty DataFrame
Columns: [children, days_employed, days_employed_fixed, age_years, education, education_id, family_status, family_status_id, gender, income_type, debt, monthly_income, monthly_income_fixed, purpose, purpose_object_id, purpose_action_id, income_type_id]
Index: []


>>Вот даже нет никакой идеи, что бы это могло быть. Можно конечно притянуть что-то за уши, но тут наверное уже нет смысла. Удаляем их.

>И переходим к нулевым стажам:

In [80]:
raw_data = raw_data.loc[raw_data['income_type'] != 'безработный']

print (raw_data[raw_data['days_employed']==0]['income_type'].value_counts())

сотрудник          1066
компаньон           497
пенсионер           383
госслужащий         144
предприниматель       1
Name: income_type, dtype: int64


>У них и стаж и доход нулевой. При этом, судя по занятости, все должны получать какой то доход. Ну может кроме предпринимателя. Т.е. это у нас пропуск данных. И те и другие данные относятся к количественным и если их восстанавливать, то нужно взять выборку по ровестникам того же пола и с тем же типом занятости, посчитать медиану для стажа и дохода и заполнить этими данными. Для начала стоит посмотреть что это за данные...

In [81]:
def get_employed_data(etype,age,gender): # функция для заполнения справочной таблицы
    temp_data = raw_data[(raw_data['income_type']==etype) & (raw_data['age_years']==age) &  (raw_data['gender']==gender) & 
                         (raw_data['days_employed'] < 0)] # так как данные о стаже у нас еще отрицательные, то отбираем их...
   
    days_min = temp_data['days_employed'].max() * -1 # и делаем из них положительные!
    days_max = temp_data['days_employed'].min() * -1
    days_med = temp_data['days_employed'].median() * -1
     
    return [days_min, days_max, days_med] #возвращаем стаж для данного пола, типа занятости и возраста мин, макс и медиану
    
employ_types = raw_data[raw_data['days_employed']==0]['income_type'].unique() #получаем список типов занятости

et_age_gender_data = pd.DataFrame(columns = ['employ_type','age','gender', 'min','max','median']) #делаем таблицу под справочник

for e_type in employ_types: # для все типов занятости    
    for x in range(16, 60): # в возрасте от 16 до 60
        gender = ['F','M']
        for g in gender: #обоих полов
            ch_data = get_employed_data(e_type, x, g) # обращаемся к функции
            data = [e_type,x,g,ch_data[0],ch_data[1],ch_data[2]] # создаем строку для записи
            et_age_gender_data.loc[et_age_gender_data.shape[0]] = data #вносим в таблицу

et_age_gender_data.dropna(inplace=True) # чистим те записи, где нет данных

print (et_age_gender_data) 

del et_age_gender_data #подчищаем

         employ_type age gender         min           max       median
94       госслужащий  19      F  509.969922    509.969922   509.969922
97       госслужащий  20      M  250.777658   1040.564490   645.671074
98       госслужащий  21      F  289.746010   1646.841004  1179.796773
99       госслужащий  21      M  476.028857   1206.015770  1178.018163
100      госслужащий  22      F  210.024134   1345.236906   465.461685
..               ...  ..    ...         ...           ...          ...
348        сотрудник  58      F  123.721457  13976.806877  1986.353728
349        сотрудник  58      M  144.618428  12747.577578  1913.139751
350        сотрудник  59      F  103.967950  16264.699501  3296.059923
351        сотрудник  59      M  236.881968  14677.902293  2363.459436
374  предприниматель  27      F  520.848083    520.848083   520.848083

[245 rows x 6 columns]


>Гадание на кофейной гуще. Нулевые данные такими и останутся!

#### Вывод 

* я считаю что данные в days_employed дискредитированы и не могут использоваться для какого то анализа
* проблема легко решается при помощи звонка тому, кто данные предоставил, но <s> это не спортивно </s> у меня нет его номера;
* рабочий стаж не та информация, которую мы должны анализировать. Он не имеет отношения к семейному положению или детям. Это сторонние данные, но если есть возможность выяснить природу ошибки и помочь коллегам, то почему нет;
* я не знаю является ли стаж официально начисленым, либо просто разность "окончание работы" - "начало работы" во втором случае все соображения о увеличении стажа за счет коэффициентов можно выкинуть в ведро. Как, видимо, и весь столбец days_employed ;)

Поэтому:

* пишем в отзывы о проблеме
* фиксим положительные стажи деля их на 30
* фиксим отрицательные стажи умножая их на (-1)
* если и испульзуем данные в дельнейшем анализе то с оглядкой

In [82]:
try:
    raw_data.insert(2,'days_employed_fixed',int) # новый столбец для поправленного стажа
except:
    print ('Столбец уже добавлен')
    
raw_data.loc[(raw_data['days_employed'] > 0),'days_employed_fixed'] = round(raw_data['days_employed']/30)
raw_data.loc[(raw_data['days_employed'] < 0),'days_employed_fixed'] = round(raw_data['days_employed']*-1)
raw_data.loc[(raw_data['days_employed'] == 0),'days_employed_fixed'] = 0

print (raw_data.info())

Столбец уже добавлен
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21328 entries, 0 to 21327
Data columns (total 17 columns):
children                21328 non-null int64
days_employed           21328 non-null float64
days_employed_fixed     21328 non-null float64
age_years               21328 non-null int64
education               21328 non-null object
education_id            21328 non-null int64
family_status           21328 non-null object
family_status_id        21328 non-null int64
gender                  21328 non-null object
income_type             21328 non-null object
debt                    21328 non-null int64
monthly_income          21328 non-null float64
monthly_income_fixed    21328 non-null int64
purpose                 21328 non-null object
purpose_object_id       21328 non-null int64
purpose_action_id       21328 non-null int64
income_type_id          21328 non-null int64
dtypes: float64(3), int64(9), object(5)
memory usage: 3.6+ MB
None


#### Проверка столбца age_years

>Похоже на проверку детей - главное чтобы не было меньше или равно 0 и не слишком большие значения

In [83]:
print ('Возраст меньше или равен нулю' , raw_data[raw_data['age_years'] <= 0]['age_years'].count())
print ('Возраст долгожителя/ей' , raw_data[raw_data['age_years'] >0]['age_years'].max())

Возраст меньше или равен нулю 100
Возраст долгожителя/ей 75


>С долгожителями все в порядке. А вот с младенцами не очень. На самом деле, нас интересуеют даже не 0, а лет 14-16, но там данные теже, я проверял. Таже сотня. 

>Итак. Возраст 0 лет это явный пропуск. Восстановить мы его не можем. Для задачи эти данные не критичны, поэтому вполне можно оставить "0" написав в рекомендации. Ну и дальше действовать помня об этом косяке. В остальном оставить как есть...

>Была мысль проверить на предмет соответствия "стаж" и "возраст", но выше я выяснил что стаж может быть и больше реального времени работы.

#### Проверка столбцов education и education_id

>Тут возможны следующие проблемы:

1. расхождение id = наименование
2. разные описания одного и того же
3. разные АйДи одного и того же

In [84]:
raw_data['education'] = raw_data['education'].str.lower() # то что напутано с регистрами было ясно еще раньше

print ('Уникальные education' ,raw_data['education'].unique())
print ('Уникальные education_id' ,raw_data['education_id'].unique())

print ('Статусы соответствуют АйДи "0" - ', raw_data[raw_data['education_id'] == 0]['education'].unique())
print ('Статусы соответствуют АйДи "1" - ', raw_data[raw_data['education_id'] == 1]['education'].unique())
print ('Статусы соответствуют АйДи "2" - ', raw_data[raw_data['education_id'] == 2]['education'].unique())
print ('Статусы соответствуют АйДи "3" - ', raw_data[raw_data['education_id'] == 3]['education'].unique())
print ('Статусы соответствуют АйДи "4" - ', raw_data[raw_data['education_id'] == 4]['education'].unique())


Уникальные education ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
Уникальные education_id [0 1 2 3 4]
Статусы соответствуют АйДи "0" -  ['высшее']
Статусы соответствуют АйДи "1" -  ['среднее']
Статусы соответствуют АйДи "2" -  ['неоконченное высшее']
Статусы соответствуют АйДи "3" -  ['начальное']
Статусы соответствуют АйДи "4" -  ['ученая степень']


>>Ну красота просто! Всегда бы так! 

#### Проверка столбцов family_status и family_status_id

>Теже соображения что и в предыдущем пункте

In [85]:
print ('Уникальные family_status' ,raw_data['family_status'].unique())
print ('Уникальные family_status_id' ,raw_data['family_status_id'].unique())

print ('Статусы соответствуют АйДи "0" - ', raw_data[raw_data['family_status_id'] == 0]['family_status'].unique())
print ('Статусы соответствуют АйДи "1" - ', raw_data[raw_data['family_status_id'] == 1]['family_status'].unique())
print ('Статусы соответствуют АйДи "2" - ', raw_data[raw_data['family_status_id'] == 2]['family_status'].unique())
print ('Статусы соответствуют АйДи "3" - ', raw_data[raw_data['family_status_id'] == 3]['family_status'].unique())
print ('Статусы соответствуют АйДи "4" - ', raw_data[raw_data['family_status_id'] == 4]['family_status'].unique())


Уникальные family_status ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
Уникальные family_status_id [0 1 2 3 4]
Статусы соответствуют АйДи "0" -  ['женат / замужем']
Статусы соответствуют АйДи "1" -  ['гражданский брак']
Статусы соответствуют АйДи "2" -  ['вдовец / вдова']
Статусы соответствуют АйДи "3" -  ['в разводе']
Статусы соответствуют АйДи "4" -  ['Не женат / не замужем']


>>Так хорошо что уже начинаю задумываться: "Что я делаю не так?!"

#### Проверка столбца gender

In [86]:
print ('Уникальные gender' ,raw_data['gender'].unique()) # список уникальных полов


Уникальные gender ['F' 'M']


In [87]:
print ('Записи с gender = XNA' , raw_data[raw_data['gender']=='XNA'])  # смотрим что с представителями третьего пола


Записи с gender = XNA Empty DataFrame
Columns: [children, days_employed, days_employed_fixed, age_years, education, education_id, family_status, family_status_id, gender, income_type, debt, monthly_income, monthly_income_fixed, purpose, purpose_object_id, purpose_action_id, income_type_id]
Index: []


In [88]:
raw_data = raw_data[raw_data['gender'] !='XNA'] # одна запись, пол определить невозможно. Можно заменить на "неопределенный", но
                                                # в данном случае проще удалить
    
print ('Проверка уникальные gender' ,raw_data['gender'].unique())

Проверка уникальные gender ['F' 'M']


#### Проверка столбца income_type

In [89]:
print ('Уникальные income_type' ,raw_data['income_type'].unique())

Уникальные income_type ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'предприниматель'
 'студент' 'в декрете']


>Все в порядке.

#### Проверка столбца debt

In [90]:
print ('Выпадающие debt' , raw_data[(raw_data['debt']!=1) & (raw_data['debt']!=0)].shape[0]) # могут быть только 0 и 1. 
#Так и есть

# можно было сделать и через unique

Выпадающие debt 0


#### Проверка столбца monthly_income

>При работе со стажем возникло подозрение, что пенсии у пенсионеров какие то слишком высокие. Не то чтобы я был против! Но, по данным Росстата средняя пенсия в январе 2018 года была 13323. Посмотрим что у нас в данных

In [92]:
print ('Минимальная пенсия (мес)', raw_data[(raw_data['income_type']=='пенсионер') & 
                                      (raw_data['monthly_income']>0)]['monthly_income'].min()) # смотрим значения больше 0
print ('Максимальная пенсия (мес)', raw_data[(raw_data['income_type']=='пенсионер')]['monthly_income'].max())

Минимальная пенсия (мес) 20667.26379327158
Максимальная пенсия (мес) 735103.2701666558


>"Ах если бы, ах если бы" Но увы, больше похоже на данные за год. Проверим по госслужащим

In [93]:
print ('Минимальная зп госслужащего (мес)', raw_data[(raw_data['income_type']=='госслужащий') & 
                                      (raw_data['monthly_income']>0)]['monthly_income'].min()) # смотрим значения больше 0
print ('Максимальная зп госслужащего (мес)', raw_data[(raw_data['income_type']=='госслужащий')]['monthly_income'].max())

Минимальная зп госслужащего (мес) 29200.077192982837
Максимальная зп госслужащего (мес) 910451.4708141336


>Опять больше похоже на годовые данные. 29 200 / 12 = 2433 в месяц. Увы, это может быть правдой. А вот 910451 в месяц. У Владимира Владимировича зп меньше! 

>Студент у нас один единственный с доходом 98201 - на него смотреть бесполезно

>Посмотрим еще сотрудников

In [94]:
print ('Минимальная зп сотрудника (мес)', raw_data[(raw_data['income_type']=='сотрудник') & 
                                      (raw_data['monthly_income']>0)]['monthly_income'].min()) # смотрим значения больше 0
print ('Максимальная зп сотрудника (мес)', raw_data[(raw_data['income_type']=='сотрудник')]['monthly_income'].max())
print ('Средняя зп сотрудника (мес)', raw_data[(raw_data['income_type']=='сотрудник')]['monthly_income'].mean())

Минимальная зп сотрудника (мес) 21367.648356486974
Максимальная зп сотрудника (мес) 1726276.0143316737
Средняя зп сотрудника (мес) 145748.5708770672


>Если поделить среднюю зп на 12 то получим 12 107. Это не соответствует статистике Росстата для средней зп по России, но гораздо ближе чем 145 286! И возможен некий перекос в наших данных - возможно за кредитами как раз обращаются люди с не самым высоким уровнем дохода

>Помню что заполнил в этом столбце 2174 пропусков нулями. Это люди с отсутствием данных о доходах и стаже.

In [95]:
print ('Записи с нулевым доходом')
print ()
#print (raw_data[(raw_data['monthly_income']==0)]) закоментировано чтобы не засорять экран. Все равно ничего интересного

Записи с нулевым доходом



>Выяснить что это за пропуски и что с ним делать не удалось. Пусть будут пока, тем более что данные не основные, а там в записях есть и дети и пропуски платежей по кредитам

>То, где есть записи делим на 12 в новый столбец

In [96]:
try:
    raw_data.insert(12,'monthly_income_fixed',int) #заводим новый столбец
except:
    print ('Столбец уже добавлен')
    
raw_data.loc[(raw_data['monthly_income'] > 0),'monthly_income_fixed'] = round(raw_data['monthly_income']/12)
raw_data.loc[(raw_data['monthly_income'] <= 0),'monthly_income_fixed'] = 0

Столбец уже добавлен


>Столбец purpose пока не трогаем - оставим на будущее до лемматизации. И переходим к ....

#### Поиск дубликатов

In [97]:
print (raw_data.duplicated().sum()) # смотрим сколько их

#Думаю что полные дубликаты в данном случае (с учетом кривости стажа и зарплаты) это точно задвоение и можно их смело удалять

raw_data = raw_data.drop_duplicates().reset_index(drop=True)
print (raw_data.duplicated().sum())

0
0


>Если бы у нас были ФИО или id или какой то другой уникальный столбец, то имело бы смысл поискать дубликаты по нему. В нашем случае можно ограничиться полными дубликатами

# Замена типа данных

>Меняем тип данных только у новых столбцов по доходу и стажу. Старые не пойдут в новый ДатаФрейм, который будет создаваться для анализа данных, поэтому пусть остаются как есть

In [100]:
raw_data['days_employed_fixed']=raw_data['days_employed_fixed'].astype('int') 
raw_data['monthly_income_fixed']=raw_data['monthly_income_fixed'].astype('int')

print (raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21328 entries, 0 to 21327
Data columns (total 17 columns):
children                21328 non-null int64
days_employed           21328 non-null float64
days_employed_fixed     21328 non-null int64
age_years               21328 non-null int64
education               21328 non-null object
education_id            21328 non-null int64
family_status           21328 non-null object
family_status_id        21328 non-null int64
gender                  21328 non-null object
income_type             21328 non-null object
debt                    21328 non-null int64
monthly_income          21328 non-null float64
monthly_income_fixed    21328 non-null int64
purpose                 21328 non-null object
purpose_object_id       21328 non-null int64
purpose_action_id       21328 non-null int64
income_type_id          21328 non-null int64
dtypes: float64(2), int64(10), object(5)
memory usage: 2.8+ MB
None


### Вывод

>Все что хотелось заменилось без ошибок.

### Лемматизация

In [101]:
print (raw_data['purpose'].value_counts()) #для начала посмотрим какие цели встречаются

свадьба                                   790
на проведение свадьбы                     763
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   648
операции с жильем                         647
операции с коммерческой недвижимостью     645
жилье                                     641
покупка жилья                             640
покупка жилья для семьи                   637
недвижимость                              631
строительство собственной недвижимости    628
операции со своей недвижимостью           623
строительство жилой недвижимости          620
покупка своего жилья                      619
строительство недвижимости                619
покупка недвижимости                      615
ремонт жилью                              603
покупка жилой недвижимости                602
на покупку своего автомобиля              504
заняться высшим образованием      

> Сначала думал в направлении лемматизировать все, потом поискать ключевые слова и по ним определить purpose_category. Но что будет, если в таблице встретиться "аренда автомобиля для свадьбы" и "аренда свадебного автомобиля"? "Постройка гаража для машины", "Покупка bmw"? В данном конкретном случае все довольно просто - тут список значений просматривается глазами. Но если строк данных будет миллион, а в purpos'ах будет такая же мешанина? Нужно...

* Пишем комментарий, что хорошо бы выбирать цели из списка фиксированных значений. 
* Смотрим на список глазами и видим что объектов у нас 4: свадьба, автомобиль, недвижимость, образование. (Что, никто не берет кредит на Samsung Galaxy S20 Ultra???) 

>Дальше чуть-чуть общих рассуждений. Раз человек хочет потратить деньги, то навряд ли он хочет продать автомобиль или недвижимость. Скорее глобальных причин 2: приобретение права собственности (возможно временного - аренда) или улучшение характиристик - ремонт, отделка и т.д. Со свадьбой и образованием еще проще. Свадьбу - сыграть, образование - получить. 

> Так у нас получается 8 пар:
> * приобретение недвижимости
* стоительство недвижимости
* ремонт недвижимости
* аренда недвижимости
* приобретение автомобиля
* ремонт автомобиля
* аренда автомобиля
* проведение свадьбы
* получение образования

> На первом этапе ограничусь этим списком. Потом посмотрю что осталось некатегоризированным и буду разбираться с ними.

>> <i>Отдельным пунктом идет поянтие "сделка". Но я приму его за аналог "покупки"</i>

Первым делом определяем объект интереса и глаголы. Для этого готовим словари синонимов, с которыми потом будем сравнивать леммы из данных. Нужно два новых столбца purpose_object_id и purpose_action_id в которых будет храниться значение полученных категорий. Кодировка при помощи степеней двойки нужна для того, чтобы если попадется что то типа "аренда автомобиля для свадьбы" мы могли легко увидеть такие задвоения. 

In [102]:
vehicle = ['автомобиль','машина', 'тачка', 'средство передвижения'] # purpose_object_id = +1
estate = ['недвижимость','квартира', 'дом', 'коттедж', 'шале', 'замок', 'землянка','жилье'] # purpose_object_id = +2
wedding = ['свадьба', 'брак','брачная церемония','венчание','помолвка'] # purpose_object_id = +4
education = ['образование','курсы','университет','институт','MBA', 'колледж'] # purpose_object_id = +8

bye = ['купить','приобретать','покупать','покупка','сделка','приобретение'] # purpose_action_id = +1
build = ['строить','строительство','возведение','создание']# purpose_action_id = +2
rent = ['аренда','арендовать','найм'] # purpose_action_id = +4
repair = ['ремонтировать','восстанавливать','ремонт'] # purpose_action_id = +8

try:
    raw_data.insert(14,'purpose_object_id',int)
except:
    print ('Столбец уже добавлен')

try:
    raw_data.insert(15,'purpose_action_id',int)
except:
    print ('Столбец уже добавлен')
    
for row in raw_data.itertuples():
    object = 0
    action = 0
    purpose = m.lemmatize(row[14])
    if (set(purpose) & set(vehicle)): # просматриваем на совпадение леммов словарю, если есть, то добавляем наш объект
        object += 1
    if (set(purpose) & set(estate)):
        object += 2
    if (set(purpose) & set(wedding)):
        object += 4
    if (set(purpose) & set(education)):
        object += 8
    
    if (set(purpose) & set(bye)): #тоже действие для действий
        action += 1
    if (set(purpose) & set(build)):
        action += 2
    if (set(purpose) & set(rent)):
        action += 4
    if (set(purpose) & set(repair)):
        action += 8
    
    raw_data.loc[row.Index, 'purpose_object_id'] = object #и вносим то что получилось в ДатаФрейм
    raw_data.loc[row.Index, 'purpose_action_id'] = action

Столбец уже добавлен
Столбец уже добавлен


>Проверяем на задвоения

In [103]:
print (raw_data['purpose_object_id'].unique())

print (raw_data['purpose_action_id'].unique())

[2 1 8 4]
[1 0 2 8]


>Все в порядке, задвоений нет. Также нет 0 в "объектах", а вот нули в "действиях" это нормально - для свадеб и образования они не нужны. Следующая проверка именно на это - у машин и недвижимости должны быть "действия", у свадеб и образования их НЕ должно быть. 

In [104]:
print ('Операции с автомобилями без действия' , raw_data[(raw_data['purpose_object_id']==1) & 
                                                         (raw_data['purpose_action_id']==0)]['purpose'].count())

print ('Операции с недвижимостью без действия' , raw_data[(raw_data['purpose_object_id']==2) & 
                                                         (raw_data['purpose_action_id']==0)]['purpose'].count())

print ('Свадьба с "действием"' , raw_data[(raw_data['purpose_object_id']==3) & 
                                                         (raw_data['purpose_action_id']!=0)]['purpose'].count())

print ('Образование с "действием"' , raw_data[(raw_data['purpose_object_id']==4) & 
                                                         (raw_data['purpose_action_id']!=0)]['purpose'].count())

Операции с автомобилями без действия 1440
Операции с недвижимостью без действия 3859
Свадьба с "действием" 0
Образование с "действием" 0


> Посмотрим подробнее что там с первыми двумя пунктами

In [105]:
print ('Операции с автомобилями без действия' , raw_data[(raw_data['purpose_object_id']==1) & 
                                                         (raw_data['purpose_action_id']==0)]['purpose'].unique())

print ('Операции с недвижимостью без действия' , raw_data[(raw_data['purpose_object_id']==2) & 
                                                         (raw_data['purpose_action_id']==0)]['purpose'].unique())

Операции с автомобилями без действия ['автомобили' 'автомобиль' 'свой автомобиль']
Операции с недвижимостью без действия ['операции с жильем' 'недвижимость'
 'операции с коммерческой недвижимостью' 'жилье'
 'операции со своей недвижимостью' 'операции с недвижимостью']


> Все это видимо "приобретение". Думаю можно безболезненно заменить все нули в action на 1

In [106]:
raw_data.loc[(raw_data['purpose_object_id']==1) & (raw_data['purpose_action_id']==0),'purpose_action_id'] = 1
raw_data.loc[(raw_data['purpose_object_id']==2) & (raw_data['purpose_action_id']==0),'purpose_action_id'] = 1

# И снова проверяем

print ('Операции с автомобилями без действия' , raw_data[(raw_data['purpose_object_id']==1) & 
                                                         (raw_data['purpose_action_id']==0)]['purpose'].count())

print ('Операции с недвижимостью без действия' , raw_data[(raw_data['purpose_object_id']==2) & 
                                                         (raw_data['purpose_action_id']==0)]['purpose'].count())

Операции с автомобилями без действия 0
Операции с недвижимостью без действия 0


### Вывод

Неожиданно подготовительный период занял много времени. Но теперь вроде все данные приведены в порядок и можно начинать категоризацию

### Категоризация данных

#### Справочники

In [107]:
print (raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21328 entries, 0 to 21327
Data columns (total 17 columns):
children                21328 non-null int64
days_employed           21328 non-null float64
days_employed_fixed     21328 non-null int64
age_years               21328 non-null int64
education               21328 non-null object
education_id            21328 non-null int64
family_status           21328 non-null object
family_status_id        21328 non-null int64
gender                  21328 non-null object
income_type             21328 non-null object
debt                    21328 non-null int64
monthly_income          21328 non-null float64
monthly_income_fixed    21328 non-null int64
purpose                 21328 non-null object
purpose_object_id       21328 non-null int64
purpose_action_id       21328 non-null int64
income_type_id          21328 non-null int64
dtypes: float64(2), int64(10), object(5)
memory usage: 2.8+ MB
None


>В результате этого действия мы, по идее, должны избавиться от все float и object (кроме purpose)

* education - в основной таблице оставляем только id. Т.к. само описание education принимает несколько фиксированных значений, то можно было бы вообще обойтись без справочника, но в учебных целях лучше его сделать
* family_status - тоже что и предыдущий пункт. Оставляем в основной таблице family_status_id
* income_type - придется создавать новый столбец с id. И так же можно было бы обойтись только id

>><i>Кстати, за время работы таблица начала занимать в памяти 3.4+ Мб, а в начале было 2. Не лучшая динамика. Посмотрим насколько объем занимаемой памяти уменьшится после создания справочников
    
> И да, паралельно со справочниками буду делать чистовую таблицу data. Вместо gendera можно сделать gender_id в котором F будет равно 1, а M - 2 Получим столбец int
    
>><i>Отдельный вопрос был с purpose'ом. В принципе, можно создать purpose_id(int) оставить в основной таблице только его, а сами purpose вынести в отдельный справочник. Но отношения все равно будут "один к одному" в отличие от других справочников. Поэтому с точки зрения организации реляционной базы данных смысла такая операция не имеет</i> 

In [108]:
education_dict = raw_data[['education', 'education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)

print ('Справочник "Образование"')
print ()
print (education_dict.head())
print ()

family_status_dict = raw_data[['family_status', 'family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)

print ('Справочник "Семейный статус"')
print ()
print (family_status_dict.head())
print ()

try:
    raw_data.insert(16,'income_type_id',int)
except:
    print ('Столбец уже добавлен')    

for row in raw_data.itertuples():
    income_type_id = 1 # по умолчанию будет сотрудник
    if row[10] == 'компаньон':
        income_type_id = 2
    if row[10] == 'пенсионер':
        income_type_id = 3
    if row[10] == 'госслужащий':
        income_type_id = 4
    if row[10] == 'предприниматель':
        income_type_id = 5
    if row[10] == 'студент':
        income_type_id = 6
    if row[10] == 'в декрете':
        income_type_id = 7   
    raw_data.loc[row.Index, 'income_type_id'] = income_type_id

income_type_dict = raw_data[['income_type', 'income_type_id']] # income_type_id создан, можно income_type выносить в стправочник
income_type_dict = income_type_dict.drop_duplicates().reset_index(drop=True)


print ('Справочник "Источник дохода"')
print ()
print (income_type_dict.head())
print ()




Справочник "Образование"

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4

Справочник "Семейный статус"

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4

Столбец уже добавлен
Справочник "Источник дохода"

       income_type  income_type_id
0        сотрудник               1
1        пенсионер               3
2        компаньон               2
3      госслужащий               4
4  предприниматель               5



In [111]:
# и создаем чистовую таблицу

data = pd.DataFrame

data = raw_data[['children','days_employed_fixed','age_years','education_id','family_status_id','gender','income_type_id',
                'debt','monthly_income_fixed','purpose_object_id','purpose_action_id','purpose']]

#try:                   
#    data.insert(6,'gender_id',int) Это была попытка создать gender_id для справочника полов, но это не привело к уменьшению
#except:
#    print ('Столбец уже добавлен')

#count = 0
#
#for row in data['gender']:
#    if row =='F':
#        data.loc[count,'gender_id'] = 1
#    else:
#        data.loc[count,'gender_id'] = 2
#    count +=1 

#data.drop(['gender'], axis=1, inplace=True)
print (data.info())

del raw_data # удаляем старый сет

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21328 entries, 0 to 21327
Data columns (total 12 columns):
children                21328 non-null int64
days_employed_fixed     21328 non-null int64
age_years               21328 non-null int64
education_id            21328 non-null int64
family_status_id        21328 non-null int64
gender                  21328 non-null object
income_type_id          21328 non-null int64
debt                    21328 non-null int64
monthly_income_fixed    21328 non-null int64
purpose_object_id       21328 non-null int64
purpose_action_id       21328 non-null int64
purpose                 21328 non-null object
dtypes: int64(10), object(2)
memory usage: 2.0+ MB
None


### Вывод

Проба заменить gender на интеджерный gender_id не оказала влияние на размер датасета, поэтому и незачем лишние движения. 

После выноса справочников из датасета размер уменьшился до прежних 2.0+ Но до выноса справочников было 3.4. Так что результат хороший. Ну и количество столбцов в датасете стало меньше 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

>Т.к. нас интересует влияет ли наличие (и количество) детей на просрочки по кредитам, то методика будет такой. Чтобы исключить половой "диморфизм" буду делать отдельно по полам. 

* Основная функция блока получат на вход пол и количество детей
* Дальше мы отбираем записи с такими же признаками и наличием просрочки и смотрим возрастные лимиты
* Отбираем ВСЕ записи с такими же признаками и с возрастными рамками из предыдущего пункта - same_age
* Считаем ratio просрочек для всего набора
* Отбираем из same_age тех, у кого количество детей соответствует заданному условию - same_age_with_children
* Отбираем тех их них, кто допустил просрочку
* И считаем ratio для них
* Прогоняем функцию для женщин с 1,2,3,4 и 5 детьми
* Повторяем для мужчин.

>>Прошу прощения за дублирования описания логики здесь в коде. Делал это для себя, чтобы не запутаться

In [121]:
def get_overdue_ratio(gender,children):
    
    first_group = data[(data['debt'] == 1) & (data['gender'] == gender) & (data['age_years']>0)
                     & (data['children']==children)] #временный сет для определения возрастных рамок
    min_age = first_group['age_years'].min()
    max_age = first_group['age_years'].max()
    
    del first_group # удаляем временный сет
    
    # Отбираем всех людей того же пола и возраста    
    same_age = data[(data['gender'] == gender) & (data['age_years']>=min_age) & (data['age_years']<=max_age)]
    
    # Считаем ratio просрочек для всего набора
    same_age_ratio =(same_age[same_age['debt']==1]['children'].count())/(same_age['children'].count())
    
    #Отбираем из same_age тех, у кого количество детей соответствует заданному условию 
    same_age_with_children = same_age[same_age['children']==children]
    
    #Отбираем тех их них, кто допустил просрочку
    same_age_with_children_overdue = same_age_with_children[same_age_with_children['debt']==1]
    
    #Считаем ratio
    with_children_ratio = (same_age_with_children_overdue['debt'].count()) / (same_age_with_children['debt'].count())
    
    
    if gender=='F':
        gender_text = 'Женщины'
    else:
        gender_text = 'Мужчины'
    print ('#######################################################################################################')
    print ()
    print ()
    print (gender_text)
    
    print ('Возрастные рамки: от', min_age, 'до', max_age)
    print ()
    print ('Всего:', same_age['children'].count())
    print ('их них допустили просрочку всего :', same_age[same_age['debt']==1]['children'].count())
    print ('что составляет {:.1%} :'. format(same_age_ratio))
    print()
    print ('Количество детей: ', children)
    print ()
    print ('всего' , same_age_with_children['debt'].count(), ', допустили просрочку'
           , same_age_with_children_overdue['debt'].count(), ', что составляет {:.1%}' . format(with_children_ratio))
    print ()
    print ()
    
    del same_age #прибираем за собой
    del same_age_with_children
    del same_age_with_children_overdue

(get_overdue_ratio('M', 1 ))
(get_overdue_ratio('M', 2 ))
(get_overdue_ratio('M', 3 ))
(get_overdue_ratio('M', 4 ))
#(get_overdue_ratio('M', 5 )) не допускают просрочек! 

(get_overdue_ratio('F', 1 ))
(get_overdue_ratio('F', 2 ))
(get_overdue_ratio('F', 3 ))
(get_overdue_ratio('F', 4 ))
#(get_overdue_ratio('M', 5 )) не допускают просрочек!

#######################################################################################################


Мужчины
Возрастные рамки: от 20 до 71

Всего: 7198
их них допустили просрочку всего : 738
что составляет 10.3% :

Количество детей:  1

всего 1716 , допустили просрочку 197 , что составляет 11.5%


#######################################################################################################


Мужчины
Возрастные рамки: от 26 до 53

Всего: 5393
их них допустили просрочку всего : 564
что составляет 10.5% :

Количество детей:  2

всего 761 , допустили просрочку 60 , что составляет 7.9%


#######################################################################################################


Мужчины
Возрастные рамки: от 27 до 47

Всего: 4329
их них допустили просрочку всего : 459
что составляет 10.6% :

Количество детей:  3

всего 122 , допустили просрочку 10 , что составляет 8.2%


##############################################################################################

### Предвывод

>На маленьких числах статистику смотреть бесполезно - очень велико влияние каждого единичного случая. Например для мужчин с 1 ребенком, каждый случай просрочки даст отклонение в сотые доли процента, для двух детей - одну десятую, для трех - 0.7, а для 4 уже 7.2% Для женщин ситуация похожая. Так что в финальном сравнении можно ограничиться "один ребенок", "два" и "три"

>То что люди с 5 детьми не допукают просрочек это конечно тоже показатель, но т.е. их вообще не много, то их рассматривать не буду

>Нужно оформить это как то покрасивей

In [129]:
def get_overdue_ratio_2(gender,children):    
    
    first_group = data[(data['debt'] == 1) & (data['gender'] == gender) & (data['age_years']>0)
                     & (data['children']==children)] #временный сет для определения возрастных рамок
    min_age = first_group['age_years'].min()
    max_age = first_group['age_years'].max()
    
    del first_group # удаляем временный сет
    
    # Отбираем всех людей того же пола и возраста    
    same_age = data[(data['gender'] == gender) & (data['age_years']>=min_age) & (data['age_years']<=max_age)]
    
    # Считаем ratio просрочек для всего набора
    same_age_ratio =(same_age[same_age['debt']==1]['children'].count())/(same_age['children'].count())
    
    #Отбираем из same_age тех, у кого количество детей соответствует заданному условию 
    same_age_with_children = same_age[same_age['children']==children]
    
    #Отбираем тех их них, кто допустил просрочку
    same_age_with_children_overdue = same_age_with_children[same_age_with_children['debt']==1]
    
    #Считаем ratio
    with_children_ratio = (same_age_with_children_overdue['debt'].count()) / (same_age_with_children['debt'].count())
    
    del same_age #прибираем за собой
    del same_age_with_children
    del same_age_with_children_overdue
    
    #Приводим вывод в красивый вид
    
    same_age_ratio = round(same_age_ratio*100,1)
    with_children_ratio = round(with_children_ratio*100,1)
    
    tstr=str(with_children_ratio) + ' (' + str(same_age_ratio) + ')'
    
    return (tstr)


date_children_overdue=pd.DataFrame(
    [
        ['мужчины',1,get_overdue_ratio_2('M', 1 )],
        ['мужчины',2,get_overdue_ratio_2('M', 2 )],
        ['мужчины',3,get_overdue_ratio_2('M', 3 )],
        ['женщины',1,get_overdue_ratio_2('F', 1 )],
        ['женщины',2,get_overdue_ratio_2('F', 2 )],
        ['женщины',3,get_overdue_ratio_2('F', 3 )]
        
    ],
    columns=['Пол','Дети','Просрочки с детьми (всего)'])

print (date_children_overdue.pivot(index='Дети', columns='Пол', values=['Просрочки с детьми (всего)']))


     Просрочки с детьми (всего)             
Пол                     женщины      мужчины
Дети                                        
1                     8.0 (7.1)  11.5 (10.3)
2                    10.8 (7.4)   7.9 (10.5)
3                     8.9 (8.2)   8.2 (10.6)


### Вывод

>В целом влияние есть

>Мужчины с детьми больше одного реже допускают просрочки, а женщины чаще. Видимо влияет то, что женщина может быть матерью-одиночкой, а мужчина с детьми это, скорее всего, полная семья. 2 ребенка это какой то особый вариант - при таком количестве детей мужчины максимально дисциплинированы, а женщины наоборот. Идеальный заемщик - мужчина с двумя детьми. Худший - мужчина с 1 ребенком. 

>Женщины более дисциплинированные плательщики.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

>Здесь пойдем другим путем - будем сравнивать процент просрочек людей в отношениях, но не со всеми того же пола и возраста, а с теми кто в отношениях не состоит. Объяснение подхода: мне интересно посмотреть какой метод окажется более верный по результатам проверки )  

* Отбираем группу с заданными характеристиками (пол и семейный статус) и с просрочкой. 
* Смотрим их возраст от и до *
* Отбираем всех того же пола этого возраста 
* Выборку делим на "замужем" family_status_id = 0 (или 1 для гражданского брака) и всех остальных. 
* В каждой подвыборке считаем процент невозвратов 

In [131]:
def get_overdue_ratio_rel(gender,family_status_id):
    
    first_group = data[(data['debt'] == 1) & (data['gender'] == gender) & (data['age_years']>0)
                     & (data['family_status_id']==family_status_id)]
    min_age = first_group['age_years'].min()
    max_age = first_group['age_years'].max()
    
    del first_group
    
    same_age = data[(data['gender'] == gender) & (data['age_years']>=min_age) & (data['age_years']<=max_age)]
    
    same_age_rel = same_age[same_age['family_status_id']==family_status_id]
    same_age_wo_rel=same_age[same_age['family_status_id']!=family_status_id]
    
    same_age_rel_overdue = same_age_rel[same_age_rel['debt']==1]
    same_age_wo_rel_overdue = same_age_wo_rel[same_age_wo_rel['debt']==1]
    
    rel_ratio = (same_age_rel_overdue['debt'].count()) / (same_age_rel['debt'].count())
    wo_rel_ratio = (same_age_wo_rel_overdue['debt'].count()) / (same_age_wo_rel['debt'].count())
    
    #Уборка!
    
    del same_age
    del same_age_rel
    del same_age_wo_rel
    del same_age_rel_overdue
    del same_age_wo_rel_overdue
    
    #Делаем красиво
    
    rel_ratio = round(rel_ratio*100,1)
    wo_rel_ratio = round(wo_rel_ratio*100,1)
    
    tstr=str(rel_ratio) + ' (' + str(wo_rel_ratio) + ')'
    
    return (tstr)

date_rel_overdue=pd.DataFrame(
    [
        ['мужчины','брак',get_overdue_ratio_rel('M', 0 )],
        ['мужчины','гражданский брак',get_overdue_ratio_rel('M', 1 )],
        ['женщины','брак',get_overdue_ratio_rel('F', 0 )],
        ['женщины','гражданский брак',get_overdue_ratio_rel('F', 1 )]
        
    ],
    columns=['Пол','Отношения','Просрочек в отношениях (без отношений)'])

print (date_rel_overdue.pivot(index='Отношения', columns='Пол', values=['Просрочек в отношениях (без отношений)']))

                 Просрочек в отношениях (без отношений)            
Пол                                             женщины     мужчины
Отношения                                                          
брак                                          6.8 (7.3)  8.8 (12.7)
гражданский брак                              8.4 (6.8)  11.8 (9.9)


### Вывод

>В браке все реже допускают просрочки. В гражданском браке наоборот. Это интересно! И опять женщины меньше просрочек допускают в принципе. Идеальный заемщик - женщина в браке. Худший - мужчина в гражданском браке

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

> Учитывая результаты двух предыдущих пунктов, опять хочется разделить по полам

* Делаем выборку пол + величина дохода с каким то шагом
* Считаем процент просрочек в каждой выборке
* Смотрим меняется ли этот процент по диапазонам

In [132]:
print ('Минимальный доход у женщин:', data[(data['gender']=='F') & (data['monthly_income_fixed']>0)]
                                           ['monthly_income_fixed'].min())
print ('Максимальный доход у женщин:', data[data['gender']=='F']
                                           ['monthly_income_fixed'].max())

print ('Минимальный доход у мужчин:', data[(data['gender']=='M') & (data['monthly_income_fixed']>0)]
                                           ['monthly_income_fixed'].min())
print ('Максимальный доход у мужчин:', data[data['gender']=='M']
                                           ['monthly_income_fixed'].max())

Минимальный доход у женщин: 1722
Максимальный доход у женщин: 142918
Минимальный доход у мужчин: 1767
Максимальный доход у мужчин: 188800


>Разделим доходы на 10 категорий Шаг определим взяв разницу между максимальной и минимальной и поделив на 10 = (188 800 - 1 722) / 10 = 18 708 .Помним что мы оперируем целыми числами! 

* I категория 0 - 20 430
* II - 20 431 - 39 138
* III - 39 139  - 57 846
* IV  - 57 847 - 76 555
* V - 76 556 - 95 264
* VI - 95 265 - 113 973
* VII - 113 974 - 132 682
* VIII - 132 683 - 151 391
* IX - 151 392 - 170 100
* X - 170 101+ 

>Дальше план:

* <s>сделать 20 датасетов по диапазонам дохода и полам</s> Одну функцию, принимающую номер диапазона и пол, и возращающую  отношение debt = 1 к общему населению диапазона
* Смотрим что получилось

In [141]:
def income_overdue(cat, gender):
    step = 20430
    min_income = step * (cat -1 )
    max_income = step * cat
        
    income_overdue_data = data[(data['gender'] == gender) & (data['monthly_income_fixed'] >= min_income)
                     & (data['monthly_income_fixed'] < max_income)] # Отбираем нужный пол внутри диапазона зарпал
                               
    total_population = income_overdue_data.shape[0] # размер всей выборки
    overdue_population = income_overdue_data['debt'].sum() # сколько с просрочкой
    
    del income_overdue_data
    
    return_string_end = str(overdue_population) + '/' + str(total_population) # вставка чтобы было видно соотношение
    
    # после первого прогона повылазила куча NaNов в резульататх, поэтому вставил проверку на 0
    if overdue_population==0: 
        return ('Просрочек нет', return_string_end)  
    
    return_string = str(round(100*overdue_population/total_population,1)) + ' %            ' + return_string_end
    
    return (return_string)

print ('Случаи задержек платежей, женщины, категория I                     ' , income_overdue(1,'F'))
print ('Случаи задержек платежей, женщины, категория II                    ' , income_overdue(2,'F'))
print ('Случаи задержек платежей, женщины, категория III                   ' , income_overdue(3,'F'))
print ('Случаи задержек платежей, женщины, категория IV                    ' , income_overdue(4,'F'))
print ('Случаи задержек платежей, женщины, категория V                     ' , income_overdue(5,'F'))
print ('Случаи задержек платежей, женщины, категория VI                    ' , income_overdue(6,'F'))
print ('Случаи задержек платежей, женщины, категория VII                   ' , income_overdue(7,'F'))
print ('Случаи задержек платежей, женщины, категория VIII                  ' , income_overdue(8,'F'))
print ('Случаи задержек платежей, женщины, категория IX                    ' , income_overdue(9,'F'))
print ('Случаи задержек платежей, женщины, категория X                     ' , income_overdue(10,'F'))

print ('Случаи задержек платежей, мужчины, категория I                     ' , income_overdue(1,'M'))
print ('Случаи задержек платежей, мужчины, категория II                    ' , income_overdue(2,'M'))
print ('Случаи задержек платежей, мужчины, категория III                   ' , income_overdue(3,'M'))
print ('Случаи задержек платежей, мужчины, категория IV                    ' , income_overdue(4,'M'))
print ('Случаи задержек платежей, мужчины, категория V                     ' , income_overdue(5,'M'))
print ('Случаи задержек платежей, мужчины, категория VI                    ' , income_overdue(6,'M'))
print ('Случаи задержек платежей, мужчины, категория VII                   ' , income_overdue(7,'M'))
print ('Случаи задержек платежей, мужчины, категория VIII                  ' , income_overdue(8,'M'))
print ('Случаи задержек платежей, мужчины, категория IX                    ' , income_overdue(9,'M'))
print ('Случаи задержек платежей, мужчины, категория X                     ' , income_overdue(10,'M'))


Случаи задержек платежей, женщины, категория I                      7.1 %            893/12563
Случаи задержек платежей, женщины, категория II                     6.3 %            90/1431
Случаи задержек платежей, женщины, категория III                    6.2 %            5/80
Случаи задержек платежей, женщины, категория IV                     14.3 %            1/7
Случаи задержек платежей, женщины, категория V                      ('Просрочек нет', '0/6')
Случаи задержек платежей, женщины, категория VI                     ('Просрочек нет', '0/3')
Случаи задержек платежей, женщины, категория VII                    ('Просрочек нет', '0/1')
Случаи задержек платежей, женщины, категория VIII                   ('Просрочек нет', '0/0')
Случаи задержек платежей, женщины, категория IX                     ('Просрочек нет', '0/0')
Случаи задержек платежей, женщины, категория X                      ('Просрочек нет', '0/0')
Случаи задержек платежей, мужчины, категория I                      10.9 %

### Предвывод

> Шаг 1. Какие то более менее достоверные данные есть только для категорий I и II. III под вопросом. Смотрим верхнюю границу категории III - грубо 58000. Переделываем все заново, но теперь в пределах диапазона от 0 до 58К

> Шаг 2 После первого шага снова выплыли диапазоны с маленькой достоверностью. Уменьшаем исследуемый диапазон до 35 000 

In [140]:
def income_overdue_2(cat, gender):
    step = 3500
    min_income = step * (cat -1 )
    max_income = step * cat
        
    income_overdue_data = data[(data['gender'] == gender) & (data['monthly_income_fixed'] >= min_income)
                     & (data['monthly_income_fixed'] < max_income)] # Отбираем нужный пол внутри диапазона зарпал
                               
    total_population = income_overdue_data.shape[0] # размер всей выборки
    overdue_population = income_overdue_data['debt'].sum() # сколько с просрочкой
    
    del income_overdue_data
    
    return_string_end = str(overdue_population) + '/' + str(total_population) # вставка чтобы было видно соотношение
    
    # после первого прогона повылазила куча NaNов в резульататх, поэтому вставил проверку на 0
    if overdue_population==0: 
        return ('Просрочек нет', return_string_end)  
    
    return_string = str(round(100*overdue_population/total_population,1)) + ' %            ' + return_string_end
    
    return (return_string)

print ('Случаи задержек платежей, женщины, категория I                     ' , income_overdue_2(1,'F'))
print ('Случаи задержек платежей, женщины, категория II                    ' , income_overdue_2(2,'F'))
print ('Случаи задержек платежей, женщины, категория III                   ' , income_overdue_2(3,'F'))
print ('Случаи задержек платежей, женщины, категория IV                    ' , income_overdue_2(4,'F'))
print ('Случаи задержек платежей, женщины, категория V                     ' , income_overdue_2(5,'F'))
print ('Случаи задержек платежей, женщины, категория VI                    ' , income_overdue_2(6,'F'))
print ('Случаи задержек платежей, женщины, категория VII                   ' , income_overdue_2(7,'F'))
print ('Случаи задержек платежей, женщины, категория VIII                  ' , income_overdue_2(8,'F'))
print ('Случаи задержек платежей, женщины, категория IX                    ' , income_overdue_2(9,'F'))
print ('Случаи задержек платежей, женщины, категория X                     ' , income_overdue_2(10,'F'))

print ('Случаи задержек платежей, мужчины, категория I                     ' , income_overdue_2(1,'M'))
print ('Случаи задержек платежей, мужчины, категория II                    ' , income_overdue_2(2,'M'))
print ('Случаи задержек платежей, мужчины, категория III                   ' , income_overdue_2(3,'M'))
print ('Случаи задержек платежей, мужчины, категория IV                    ' , income_overdue_2(4,'M'))
print ('Случаи задержек платежей, мужчины, категория V                     ' , income_overdue_2(5,'M'))
print ('Случаи задержек платежей, мужчины, категория VI                    ' , income_overdue_2(6,'M'))
print ('Случаи задержек платежей, мужчины, категория VII                   ' , income_overdue_2(7,'M'))
print ('Случаи задержек платежей, мужчины, категория VIII                  ' , income_overdue_2(8,'M'))
print ('Случаи задержек платежей, мужчины, категория IX                    ' , income_overdue_2(9,'M'))
print ('Случаи задержек платежей, мужчины, категория X                     ' , income_overdue_2(10,'M'))

Случаи задержек платежей, женщины, категория I                      6.9 %            107/1549
Случаи задержек платежей, женщины, категория II                     6.8 %            142/2077
Случаи задержек платежей, женщины, категория III                    7.6 %            264/3477
Случаи задержек платежей, женщины, категория IV                     7.4 %            215/2889
Случаи задержек платежей, женщины, категория V                      6.7 %            115/1721
Случаи задержек платежей, женщины, категория VI                     5.9 %            56/943
Случаи задержек платежей, женщины, категория VII                    5.6 %            36/640
Случаи задержек платежей, женщины, категория VIII                   8.0 %            24/300
Случаи задержек платежей, женщины, категория IX                     8.3 %            15/181
Случаи задержек платежей, женщины, категория X                      3.4 %            4/118
Случаи задержек платежей, мужчины, категория I                      10.

### Вывод

>Единственная четко выделяемая зависимость - у мужчин с низким доходом дисциплина самая низкая. Связано, скорее всего, с характером поведения молодых (мало зарабатывающих) мужчин. Когда важно "держать марку любой ценой" покупая то, что не по карману. А денег на возврат не всегда хватает. 

- Как разные цели кредита влияют на его возврат в срок?

>Алгорит:

* Выбираем всех заемщиков женского пола -> Смотрим количество просрочек -> Считаем коеэффициен f_overdue_ratio
* Выбираем женщин с конкретной целью (автомобиль) -> Смотрим количество просрочек -> Считаем коеэффициен f_overdue_ratio_1
* Повторяем для все целей. Для недвижимости будут 2_1 - покупка, 2_2 - строительство, 2_8 - ремонт 
* Все тоже самое для мужчин

>>Аренды у нас нет вообще, поэтому не рассматриваем

In [99]:
f_data = data[data['gender'] == 'F']
m_data = data[data['gender'] == 'M']

f_overdue_ratio = f_data[f_data['debt']==1]['debt'].count() / f_data.shape[0] 
m_overdue_ratio = m_data[m_data['debt']==1]['debt'].count() / m_data.shape[0] 

f_data_1_1 = f_data[(f_data['purpose_object_id']==1) & (f_data['purpose_action_id']==1)]
m_data_1_1 = m_data[(m_data['purpose_object_id']==1) & (m_data['purpose_action_id']==1)]

f_overdue_ratio_1_1 = f_data_1_1[f_data_1_1['debt']==1]['debt'].count() / f_data_1_1.shape[0] 
m_overdue_ratio_1_1 = m_data_1_1[m_data_1_1['debt']==1]['debt'].count() / m_data_1_1.shape[0] 

del f_data_1_1 # подчищаем отработанные датафреймы
del m_data_1_1 # подчищаем отработанные датафреймы

f_data_2_1 = f_data[(f_data['purpose_object_id']==2) & (f_data['purpose_action_id']==1)]
m_data_2_1 = m_data[(m_data['purpose_object_id']==2) & (m_data['purpose_action_id']==1)]

f_overdue_ratio_2_1 = f_data_2_1[f_data_2_1['debt']==1]['debt'].count() / f_data_2_1.shape[0] 
m_overdue_ratio_2_1 = m_data_2_1[m_data_2_1['debt']==1]['debt'].count() / m_data_2_1.shape[0] 

del f_data_2_1 # подчищаем отработанные датафреймы
del m_data_2_1 # подчищаем отработанные датафреймы

f_data_2_2 = f_data[(f_data['purpose_object_id']==2) & (f_data['purpose_action_id']==2)]
m_data_2_2 = m_data[(m_data['purpose_object_id']==2) & (m_data['purpose_action_id']==2)]

f_overdue_ratio_2_2 = f_data_2_2[f_data_2_2['debt']==1]['debt'].count() / f_data_2_2.shape[0] 
m_overdue_ratio_2_2 = m_data_2_2[m_data_2_2['debt']==1]['debt'].count() / m_data_2_2.shape[0] 

del f_data_2_2 # подчищаем отработанные датафреймы
del m_data_2_2 # подчищаем отработанные датафреймы

f_data_2_8 = f_data[(f_data['purpose_object_id']==2) & (f_data['purpose_action_id']==8)]
m_data_2_8 = m_data[(m_data['purpose_object_id']==2) & (m_data['purpose_action_id']==8)]

f_overdue_ratio_2_8 = f_data_2_8[f_data_2_8['debt']==1]['debt'].count() / f_data_2_8.shape[0] 
m_overdue_ratio_2_8 = m_data_2_8[m_data_2_8['debt']==1]['debt'].count() / m_data_2_8.shape[0] 

del f_data_2_8 # подчищаем отработанные датафреймы
del m_data_2_8 # подчищаем отработанные датафреймы

f_data_4 = f_data[(f_data['purpose_object_id']==4)]
m_data_4 = m_data[(m_data['purpose_object_id']==4)]

f_overdue_ratio_4 = f_data_4[f_data_4['debt']==1]['debt'].count() / f_data_4.shape[0] 
m_overdue_ratio_4 = m_data_4[m_data_4['debt']==1]['debt'].count() / m_data_4.shape[0] 

del f_data_4 # подчищаем отработанные датафреймы
del m_data_4 # подчищаем отработанные датафреймы

f_data_8 = f_data[(f_data['purpose_object_id']==8)]
m_data_8 = m_data[(m_data['purpose_object_id']==8)]

f_overdue_ratio_8 = f_data_8[f_data_8['debt']==1]['debt'].count() / f_data_8.shape[0] 
m_overdue_ratio_8 = m_data_8[m_data_8['debt']==1]['debt'].count() / m_data_8.shape[0] 

del f_data_8 # подчищаем отработанные датафреймы
del m_data_8 # подчищаем отработанные датафреймы
                  
del f_data # подчищаем отработанные датафреймы
del m_data # подчищаем отработанные датафреймы

print()
print('################ ОБЩИЕ ДАННЫЕ ###################')
print()
print ('Процент просрочек у женщин: {:.1%}' . format(f_overdue_ratio))
print ('Процент просрочек у мужчин: {:.1%}' . format(m_overdue_ratio))
print()
print()
print('############# ПОКУПКА АВТОМОБИЛЯ ##############')
print()
print ('Процент просрочек у женщин: {:.1%}' . format(f_overdue_ratio_1_1))
print ('Процент просрочек у мужчин: {:.1%}' . format(m_overdue_ratio_1_1))
print()
print('############# ПОКУПКА НЕДВИЖИМОСТИ ##############')
print()
print ('Процент просрочек у женщин: {:.1%}' . format(f_overdue_ratio_2_1))
print ('Процент просрочек у мужчин: {:.1%}' . format(m_overdue_ratio_2_1))
print()
print('############# ПОCТРОЙКА НЕДВИЖИМОСТИ ##############')
print()
print ('Процент просрочек у женщин: {:.1%}' . format(f_overdue_ratio_2_2))
print ('Процент просрочек у мужчин: {:.1%}' . format(m_overdue_ratio_2_2))
print()
print('############# РЕМОНТ НЕДВИЖИМОСТИ ##############')
print()
print ('Процент просрочек у женщин: {:.1%}' . format(f_overdue_ratio_2_8))
print ('Процент просрочек у мужчин: {:.1%}' . format(m_overdue_ratio_2_8))
print()
print('############# СВАДЬБА ##############')
print()
print ('Процент просрочек у женщин: {:.1%}' . format(f_overdue_ratio_4))
print ('Процент просрочек у мужчин: {:.1%}' . format(m_overdue_ratio_4))
print()
print('############# ОБРАЗОВАНИЕ ##############')
print()
print ('Процент просрочек у женщин: {:.1%}' . format(f_overdue_ratio_8))
print ('Процент просрочек у мужчин: {:.1%}' . format(m_overdue_ratio_8))
print()


################ ОБЩИЕ ДАННЫЕ ###################

Процент просрочек у женщин: 7.0%
Процент просрочек у мужчин: 10.3%


############# ПОКУПКА АВТОМОБИЛЯ ##############

Процент просрочек у женщин: 8.2%
Процент просрочек у мужчин: 11.7%

############# ПОКУПКА НЕДВИЖИМОСТИ ##############

Процент просрочек у женщин: 6.4%
Процент просрочек у мужчин: 8.9%

############# ПОCТРОЙКА НЕДВИЖИМОСТИ ##############

Процент просрочек у женщин: 6.0%
Процент просрочек у мужчин: 10.9%

############# РЕМОНТ НЕДВИЖИМОСТИ ##############

Процент просрочек у женщин: 5.9%
Процент просрочек у мужчин: 5.6%

############# СВАДЬБА ##############

Процент просрочек у женщин: 6.9%
Процент просрочек у мужчин: 10.0%

############# ОБРАЗОВАНИЕ ##############

Процент просрочек у женщин: 7.8%
Процент просрочек у мужчин: 12.2%



### Вывод

>И опять женщины дисциплинированней мужчин! Одно исключение - ремонт недвижимости. Но я думаю что это из разряда погрешности: данная причина кредитования наименнее популярна и там велико воздействие случайных выбросов.  

> Что касается различий по целям, то я бы не сказал что они какие то значимые. 

### Шаг 4. Общий вывод

* Женщины дисциплиннированей мужчин!
* Данные нужно лучше готовит! ))

Пожелания по ходу проекта:

	1. При выгрузке данных переименовать столбец dob_years в age_years и total_income в monthly_income. Так больше соответствует сути
	2. days_employed - в выгрузке присутствуют отрицательные данные (невозможно) и числа с плавающей точкой (должны быть целые)
	3. total_income  - в выгрузке присутствуют числа с плавающей точкой (возможно, но для наших целей достаточно точности в рублях) Лучше переделать на int
	4. days_employed - в выгрузке сильно напутано с данными - отрицательные значения стажа, чудовищные положительные числа (возможно лишнее умножение на 30), нулевые стажи - в доработку! 
	5. Столбец children - 76 записей с 20 детьми и 47 записей с отрицательным числом. Итого 123 записи. Обратить внимание!
	6. В выгрузке у 100 человек возраст 0
	7. В столбце education путаница с регистрами
	8. В столбе monthly_income (тот, который до переименования был total_income) указаны странные данные. Во-первых они с кучей (9 - 10) знаков после запятой. Во вторых по размеру они скорее соответствуют годовым доходам. Может total в старом названии столбца и означал "годовой"? Но в описании "ежемесячный доход"
    9. Цели кредита должны выбираться из списка фиксированных значений, чтобы не было путаницы типа "ремонт жилью"!